In [1]:
import networkx as nx
import heapq as hq
from copy import deepcopy

# For some reason pylance doesn't like these imports, but they work fine
import KnowledgeGraph as kg  # type: ignore
import ConceptNode as cn  # type: ignore
from MultiDistance import MultiDistance, multimin, sum_pareto_distance  # type: ignore
%load_ext snakeviz

In [2]:
G_L0 = nx.DiGraph()
G_L1 = nx.DiGraph()
G_L2 = nx.DiGraph()

G_L0.add_edges_from([
    ('A', 'B', dict(weight=1)),
    ('A', 'E', dict(weight=5)),
    ('A', 'D', dict(weight=1)),
    ('B', 'C', dict(weight=1)),
    ('D', 'E', dict(weight=1)),
    ('C', 'E', dict(weight=1)),
    ('E', 'F', dict(weight=1))])

G_L1.add_edges_from([
    ('A', 'C', dict(weight=1)),
    ('C', 'B', dict(weight=1)),
    ('B', 'D', dict(weight=1)),
    ('B', 'F', dict(weight=1)),
    ('D', 'E', dict(weight=1))])

G_L2.add_edges_from([
    ('A', 'B', dict(weight=1)),
    ('C', 'D', dict(weight=1)),
    ('E', 'F', dict(weight=1))])

layers = {'L0': G_L0, 'L1': G_L1, 'L2': G_L2}
#layers = {'L0': G_L0, 'L1': G_L1}
KG = kg.KnowledgeGraph(layers)


In [3]:
dist = KG.pareto_shortest_distances(KG.nodes[('A', 'L0')],cut_by_neighbors=False)
for k, v in sorted(dist.items()):
    print(k, v)


('A', 'L0') [{}]
('A', 'L1') [{}]
('A', 'L2') [{}]
('B', 'L0') [{('L2', 'L2'): 1}, {('L1', 'L1'): 2}, {('L0', 'L0'): 1}]
('B', 'L1') [{('L2', 'L2'): 1}, {('L1', 'L1'): 2}, {('L0', 'L0'): 1}]
('B', 'L2') [{('L2', 'L2'): 1}, {('L1', 'L1'): 2}, {('L0', 'L0'): 1}]
('C', 'L0') [{('L2', 'L2'): 1, ('L0', 'L0'): 1}, {('L1', 'L1'): 1}, {('L0', 'L0'): 2}]
('C', 'L1') [{('L2', 'L2'): 1, ('L0', 'L0'): 1}, {('L0', 'L0'): 2}, {('L1', 'L1'): 1}]
('C', 'L2') [{('L2', 'L2'): 1, ('L0', 'L0'): 1}, {('L0', 'L0'): 2}, {('L1', 'L1'): 1}]
('D', 'L0') [{('L1', 'L1'): 3}, {('L1', 'L1'): 1, ('L2', 'L2'): 1}, {('L0', 'L0'): 1}]
('D', 'L1') [{('L1', 'L1'): 3}, {('L1', 'L1'): 1, ('L2', 'L2'): 1}, {('L0', 'L0'): 1}]
('D', 'L2') [{('L1', 'L1'): 3}, {('L1', 'L1'): 1, ('L2', 'L2'): 1}, {('L0', 'L0'): 1}]
('E', 'L0') [{('L1', 'L1'): 4}, {('L1', 'L1'): 2, ('L2', 'L2'): 1}, {('L0', 'L0'): 1, ('L1', 'L1'): 1}, {('L0', 'L0'): 2}]
('E', 'L1') [{('L1', 'L1'): 4}, {('L1', 'L1'): 2, ('L2', 'L2'): 1}, {('L0', 'L0'): 2}, {('L0',

In [4]:
dist[('C', 'L0')]


[{('L2', 'L2'): 1, ('L0', 'L0'): 1}, {('L1', 'L1'): 1}, {('L0', 'L0'): 2}]

In [5]:
all_dists = {k: v for k, v in KG.all_pairs_pareto_distance(cut_by_neighbors=False)}


In [6]:
all_dists


{('A', 'L0'): {('A', 'L0'): [{}],
  ('A', 'L1'): [{}],
  ('D', 'L0'): [{('L1', 'L1'): 3},
   {('L1', 'L1'): 1, ('L2', 'L2'): 1},
   {('L0', 'L0'): 1}],
  ('B', 'L0'): [{('L2', 'L2'): 1}, {('L1', 'L1'): 2}, {('L0', 'L0'): 1}],
  ('C', 'L1'): [{('L2', 'L2'): 1, ('L0', 'L0'): 1},
   {('L0', 'L0'): 2},
   {('L1', 'L1'): 1}],
  ('D', 'L1'): [{('L1', 'L1'): 3},
   {('L1', 'L1'): 1, ('L2', 'L2'): 1},
   {('L0', 'L0'): 1}],
  ('B', 'L2'): [{('L2', 'L2'): 1}, {('L1', 'L1'): 2}, {('L0', 'L0'): 1}],
  ('E', 'L0'): [{('L1', 'L1'): 4},
   {('L1', 'L1'): 2, ('L2', 'L2'): 1},
   {('L0', 'L0'): 1, ('L1', 'L1'): 1},
   {('L0', 'L0'): 2}],
  ('C', 'L0'): [{('L2', 'L2'): 1, ('L0', 'L0'): 1},
   {('L1', 'L1'): 1},
   {('L0', 'L0'): 2}],
  ('E', 'L1'): [{('L1', 'L1'): 4},
   {('L1', 'L1'): 2, ('L2', 'L2'): 1},
   {('L0', 'L0'): 2},
   {('L0', 'L0'): 1, ('L1', 'L1'): 1}],
  ('C', 'L2'): [{('L2', 'L2'): 1, ('L0', 'L0'): 1},
   {('L0', 'L0'): 2},
   {('L1', 'L1'): 1}],
  ('E', 'L2'): [{('L1', 'L1'): 4},
   {(

In [7]:
cdist = {k[0]: {vk[0]: vv for vk, vv in v.items()}
         for k, v in all_dists.items()}


In [8]:
cdist


{'A': {'A': [{}],
  'B': [{('L1', 'L1'): 2}, {('L0', 'L0'): 1}, {('L2', 'L2'): 1}],
  'C': [{('L1', 'L1'): 1},
   {('L0', 'L0'): 2},
   {('L2', 'L2'): 1, ('L0', 'L0'): 1}],
  'D': [{('L1', 'L1'): 3},
   {('L0', 'L0'): 1},
   {('L2', 'L2'): 1, ('L1', 'L1'): 1}],
  'E': [{('L1', 'L1'): 4},
   {('L0', 'L0'): 1, ('L1', 'L1'): 1},
   {('L0', 'L0'): 2},
   {('L2', 'L2'): 1, ('L1', 'L1'): 2}],
  'F': [{('L1', 'L1'): 3},
   {('L0', 'L0'): 1, ('L1', 'L1'): 1},
   {('L0', 'L0'): 2, ('L2', 'L2'): 1},
   {('L0', 'L0'): 3},
   {('L2', 'L2'): 1, ('L1', 'L1'): 1}]},
 'B': {'B': [{}],
  'C': [{('L0', 'L0'): 1}],
  'D': [{('L0', 'L0'): 1, ('L2', 'L2'): 1}, {('L1', 'L1'): 1}],
  'E': [{('L1', 'L1'): 2},
   {('L1', 'L1'): 1, ('L0', 'L0'): 1},
   {('L0', 'L0'): 2}],
  'F': [{('L0', 'L0'): 2, ('L2', 'L2'): 1},
   {('L0', 'L0'): 3},
   {('L1', 'L1'): 1}]},
 'E': {'E': [{}], 'F': [{('L0', 'L0'): 1}, {('L2', 'L2'): 1}]},
 'D': {'D': [{}],
  'E': [{('L0', 'L0'): 1}, {('L1', 'L1'): 1}],
  'F': [{('L1', 'L1'): 1

In [9]:
cdist['A']['E']


[{('L1', 'L1'): 4},
 {('L0', 'L0'): 1, ('L1', 'L1'): 1},
 {('L0', 'L0'): 2},
 {('L2', 'L2'): 1, ('L1', 'L1'): 2}]

In [10]:
closure = KG.pareto_distance_closure(start_layer='L0',cut_by_neighbors=False)


In [11]:
closure['A']['E']


[{('L1', 'L1'): 4},
 {('L1', 'L1'): 2, ('L2', 'L2'): 1},
 {('L0', 'L0'): 1, ('L1', 'L1'): 1},
 {('L0', 'L0'): 2}]

In [12]:
removed_edges = KG.pareto_backbone_removed_edges(closure)


In [13]:
removed_edges


{(('A', 'L0'), ('E', 'L0'))}

In [14]:
layer_weights = {
    ('L0', 'L0'): 2,
    ('L1', 'L1'): 4,
    ('L2', 'L2'): 1,
}

removed_edges_weighted = KG.weighted_backbone_removed_edges(closure=closure,
                                                            layer_weights=layer_weights)


In [15]:
removed_edges_weighted


{(('A', 'L0'), ('B', 'L0')),
 (('A', 'L0'), ('E', 'L0')),
 (('A', 'L1'), ('C', 'L1')),
 (('B', 'L1'), ('D', 'L1')),
 (('D', 'L1'), ('E', 'L1')),
 (('E', 'L0'), ('F', 'L0'))}

In [20]:
ER0 = nx.erdos_renyi_graph(300, .05)
ER1 = nx.erdos_renyi_graph(60, 0.3)
ER2 = nx.erdos_renyi_graph(60, 0.3)
ER3 = nx.erdos_renyi_graph(100, 0.3)
ER4 = nx.erdos_renyi_graph(100, 0.3)
ER5 = nx.erdos_renyi_graph(100, 0.3)

layersER = {
    'L0': ER0,
    #'L1':ER1,
    #'L2':ER2,
    #'L3':ER3,
    # 'L4':ER4,
    # 'L5':ER5,
}


for ER in layersER.values():
    for (u, v) in ER.edges():
        ER.edges[(u, v)]['weight'] = u+v


ERKG = kg.KnowledgeGraph(layersER)


In [30]:
from multiprocessing import Pool
import mp_for_nb


arglist = mp_for_nb.make_arglist(ERKG,'L0')

with Pool() as pool:
    results = pool.imap_unordered(mp_for_nb.appd,arglist)
    %snakeviz -t APP_mp = list(results)

 
*** Profile stats marshalled to file 'C:\\Users\\jcroz\\AppData\\Local\\Temp\\tmpxjzyt4su'.
Opening SnakeViz in a new tab...


In [22]:
%snakeviz -t APP=list(ERKG.all_pairs_pareto_distance(start_layer='L0',cut_by_neighbors=False,depth_cut=None))

 
*** Profile stats marshalled to file 'C:\\Users\\jcroz\\AppData\\Local\\Temp\\tmpcsqgnno5'.
Opening SnakeViz in a new tab...


In [23]:
%snakeviz -t APD=list(nx.all_pairs_dijkstra(ER0))

 
*** Profile stats marshalled to file 'C:\\Users\\jcroz\\AppData\\Local\\Temp\\tmp9_eibuhe'.
Opening SnakeViz in a new tab...


In [34]:
APDs=[x[1][0] for x in APD]
APPs=[{k[0]:list(v[0].items())[0][1] for k,v in x[1].items() if len(v[0])>0 } for x in APP]
# account for convention differences in self-distance
for i,d in enumerate(APPs):
    d[i]=0

    
print(APDs==APPs)

True
True


In [46]:
APP_mp_dict = dict(APP_mp)
APP_dict = dict(APP)
APP_mp_dict==APP_dict